# Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl

# Preparing TPU

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Loading Data

In [ ]:
from tensorflow.keras.datasets import cifar100

(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='coarse')

In [ ]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

# Resizing images

In [ ]:
import cv2
X_train = np.array([cv2.resize(img, (224, 224)) for img in X_train])

In [ ]:
X_test = np.array([cv2.resize(img, (224, 224)) for img in X_test])

# Encoding labels

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
y_train=enc.fit_transform(y_train).toarray().astype(int)
y_test=enc.transform(y_test).toarray().astype(int)


print(y_train.shape)
print(y_train[0])

# Preparing mobileNet_V2's Preprocessing

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Transfer Learning from MobileNet_V2

In [ ]:
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)
MobileNet = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
MobileNet.summary()

# Using MobileNet_V2 in my architecture

I will build two architectures to see the improvement that I do on the first one

In [ ]:
def transfer_from_MN(image_shape=IMG_SIZE):

    input_shape = image_shape + (3,)

    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False,
                                                   weights='imagenet')

    # freeze the base model by making it non trainable
    base_model.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape)

    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(inputs)

    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False)

    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x)
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)

    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tfl.Dense(units=20, activation='softmax')(x)

    ### END CODE HERE

    model = tf.keras.Model(inputs, outputs)

    return model

In [ ]:
def transfer_from_MN_2(image_shape=IMG_SIZE):

    input_shape = image_shape + (3,)

    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False,
                                                   weights='imagenet')

    # freeze the base model by making it non trainable
    base_model.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape)

    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(inputs)

    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False)

    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x)
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)

    x = tfl.Dense(units=100)(x)
    x = tfl.LeakyReLU()(x)
    x = tfl.Dropout(0.2)(x)
    x = tfl.Dense(units=100)(x)
    x = tfl.LeakyReLU()(x)
    x = tfl.Dropout(0.2)(x)
    x = tfl.Dense(units=100)(x)
    x = tfl.LeakyReLU()(x)
    x = tfl.Dropout(0.2)(x)
    x = tfl.Dense(units=100)(x)
    x = tfl.LeakyReLU()(x)
    x = tfl.Dropout(0.2)(x)
    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tfl.Dense(units=20, activation='softmax')(x)

    ### END CODE HERE

    model = tf.keras.Model(inputs, outputs)

    return model

# Training and evaluating the model

In [ ]:
# instantiating the model in the strategy scope creates the model on the TPU
with strategy.scope():
    CIFAR_Recognizer_1 = transfer_from_MN(IMG_SIZE)
    CIFAR_Recognizer_1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    CIFAR_Recognizer_2 = transfer_from_MN_2(IMG_SIZE)
    CIFAR_Recognizer_2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(16 * strategy.num_replicas_in_sync)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16 * strategy.num_replicas_in_sync)
initial_epochs = 50

In [ ]:
history1 = CIFAR_Recognizer_1.fit(train_dataset, validation_data=test_dataset, batch_size=16 * strategy.num_replicas_in_sync,epochs=initial_epochs,shuffle=True)

In [ ]:
history2 = CIFAR_Recognizer_2.fit(train_dataset, validation_data=test_dataset, batch_size=16 * strategy.num_replicas_in_sync,epochs=initial_epochs,shuffle=True)

# Models' performance¶

In [ ]:
plt.plot(history1.history["accuracy"])

In [ ]:
plt.plot(history2.history["accuracy"])

# Saving the model¶

In [ ]:
tf.keras.models.save_model(CIFAR_Recognizer_2, '/kaggle/working/model.h5')

# Downloading the model

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'model.zip'):
    """
    zip all the files in a directory

    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory

    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'

    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [ ]:
zip_dir()